# Evolutionary Analysis Using Likelihood

## Specifying substitution models

### The available pre-defined substitution models

In cases where code takes a substitution model as an argument, you can use the value under "Abbreviation" as a string.

In [1]:
from cogent3 import available_models

available_models()

nucleotide,JC69,Jukes and Cantor's 1969 model
nucleotide,K80,Kimura 1980
nucleotide,F81,Felsenstein's 1981 model
nucleotide,HKY85,"Hasegawa, Kishino and Yanamo 1985 model"
nucleotide,TN93,Tamura and Nei 1993 model
nucleotide,GTR,General Time Reversible nucleotide substitution model.
nucleotide,ssGN,"strand-symmetric general Markov nucleotide (non-stationary, non-reversible). Kaehler, 2017, Journal of Theoretical Biology 420: 144–51"
nucleotide,GN,"General Markov Nucleotide (non-stationary, non-reversible). Kaehler, Yap, Zhang, Huttley, 2015, Sys Biol 64 (2): 281–93"
nucleotide,BH,Barry and Hartigan Discrete Time substitution model Barry and Hartigan 1987. Biometrics 43: 261–76.
nucleotide,DT,"Discrete Time substitution model (non-stationary, non-reversible). motif_length=2 makes this a dinucleotide model, motif_length=3 a trinucleotide model."
codon,CNFGTR,"Conditional nucleotide frequency codon substitution model, GTR variant (with params analagous to the nucleotide GTR model). Yap, Lindsay, Easteal and Huttley, 2010, Mol Biol Evol 27: 726-734"


### Getting a substitution model with ``get_model()``

In [2]:
from cogent3.evolve.models import get_model

hky = get_model("HKY85")
print(hky)


TimeReversibleNucleotide ( name = 'HKY85'; type = 'None'; params = ['kappa']; number of motifs = 4; motifs = ['T', 'C', 'A', 'G'])



### Rate heterogeneity models

We illustrate this for the gamma distributed case using examples of the canned models displayed above. Creating rate heterogeneity variants of the canned models can be done by using optional arguments that get passed to the substitution model class.

#### For nucleotide

We specify a general time reversible nucleotide model with gamma distributed rate heterogeneity.

In [3]:
from cogent3.evolve.models import get_model

sub_mod = get_model("GTR", with_rate=True, distribution='gamma')

print(sub_mod)


TimeReversibleNucleotide ( name = 'GTR'; type = 'None'; params = ['A/C', 'A/G', 'A/T', 'C/G', 'C/T']; number of motifs = 4; motifs = ['T', 'C', 'A', 'G'])



#### For codon

We specify a conditional nucleotide frequency codon model with nucleotide general time reversible parameters and a parameter for the ratio of nonsynonymous to synonymous substitutions (omega) with gamma distributed rate heterogeneity.

In [4]:
from cogent3.evolve.models import get_model

sub_mod = get_model("CNFGTR", with_rate=True, distribution='gamma')

print(sub_mod)


TimeReversibleCodon ( name = 'CNFGTR'; type = 'None'; params = ['A/C', 'A/G', 'A/T', 'C/G', 'C/T', 'omega']; number of motifs = 61; motifs = ['TTT', 'TTC', 'TTA', 'TTG', 'TCT', 'TCC', 'TCA', 'TCG', 'TAT', 'TAC', 'TGT', 'TGC', 'TGG', 'CTT', 'CTC', 'CTA', 'CTG', 'CCT', 'CCC', 'CCA', 'CCG', 'CAT', 'CAC', 'CAA', 'CAG', 'CGT', 'CGC', 'CGA', 'CGG', 'ATT', 'ATC', 'ATA', 'ATG', 'ACT', 'ACC', 'ACA', 'ACG', 'AAT', 'AAC', 'AAA', 'AAG', 'AGT', 'AGC', 'AGA', 'AGG', 'GTT', 'GTC', 'GTA', 'GTG', 'GCT', 'GCC', 'GCA', 'GCG', 'GAT', 'GAC', 'GAA', 'GAG', 'GGT', 'GGC', 'GGA', 'GGG'])



#### For protein

We specify a Jones, Taylor and Thornton 1992 empirical protein substitution model with gamma distributed rate heterogeneity.

In [5]:
from cogent3.evolve.models import get_model

sub_mod = get_model("JTT92", with_rate=True, distribution='gamma')

print(sub_mod)


Empirical ( name = 'JTT92'; type = 'None'; number of motifs = 20; motifs = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])



## Making a likelihood function

You start by specifying a substitution model and use that to construct a likelihood function for a specific tree.

In [6]:
from cogent3 import make_tree
from cogent3.evolve.models import get_model

sub_mod = get_model("F81")
tree = make_tree('(a,b,(c,d))')

lf = sub_mod.make_likelihood_function(tree)

### Providing an alignment to a likelihood function

You need to load an alignment and then provide it a likelihood function. I construct very simple trees and alignments for this example.

In [7]:
from cogent3 import make_tree, make_aligned_seqs
from cogent3.evolve.models import get_model
sub_mod = get_model("F81")
tree = make_tree('(a,b,(c,d))')
lf = sub_mod.make_likelihood_function(tree)
aln = make_aligned_seqs([('a', 'ACGT'), ('b', 'AC-T'), ('c', 'ACGT'),
                     ('d', 'AC-T')])
lf.set_alignment(aln)

### Scoping parameters on trees -- time heterogeneous models

For many evolutionary analyses, it's desirable to allow different branches on a tree to have different values of a parameter. We show this for a simple codon model case here where we want the great apes (the clade that includes human and orangutan) to have a different value of the ratio of nonsynonymous to synonymous substitutions. This parameter is identified in the precanned ``CNFGTR`` model as ``omega``.

In [8]:
from cogent3 import load_tree
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')

print(tree.ascii_art())

          /-Galago
         |
-root----|--HowlerMon
         |
         |          /-Rhesus
          \edge.3--|
                   |          /-Orangutan
                    \edge.2--|
                             |          /-Gorilla
                              \edge.1--|
                                       |          /-Human
                                        \edge.0--|
                                                  \-Chimpanzee


In [9]:
sm = get_model("CNFGTR")
lf = sm.make_likelihood_function(tree, digits=2)
lf.set_param_rule('omega', tip_names=['Human', 'Orangutan'], outgroup_name='Galago', clade=True, init=0.5)

We've set an *initial* value for this clade so that the edges affected by this rule are evident below.

In [10]:
lf

Likelihood function statistics
number of free parameters = 78
====================================
 A/C     A/G     A/T     C/G     C/T
------------------------------------
1.00    1.00    1.00    1.00    1.00
------------------------------------
=======================================
      edge    parent    length    omega
---------------------------------------
    Galago      root      1.00     1.00
 HowlerMon      root      1.00     1.00
    Rhesus    edge.3      1.00     1.00
 Orangutan    edge.2      1.00     0.50
   Gorilla    edge.1      1.00     0.50
     Human    edge.0      1.00     0.50
Chimpanzee    edge.0      1.00     0.50
    edge.0    edge.1      1.00     0.50
    edge.1    edge.2      1.00     0.50
    edge.2    edge.3      1.00     1.00
    edge.3      root      1.00     1.00
---------------------------------------
============================================================================
 AAA     AAC     AAG     AAT     ACA     ACC     ACG     ACT     AGA     AGC
----------------------------------------------------------------------------
0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02
----------------------------------------------------------------------------

continued: 
============================================================================
 AGG     AGT     ATA     ATC     ATG     ATT     CAA     CAC     CAG     CAT
----------------------------------------------------------------------------
0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02
----------------------------------------------------------------------------

continued: 
============================================================================
 CCA     CCC     CCG     CCT     CGA     CGC     CGG     CGT     CTA     CTC
----------------------------------------------------------------------------
0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02
----------------------------------------------------------------------------

continued: 
============================================================================
 CTG     CTT     GAA     GAC     GAG     GAT     GCA     GCC     GCG     GCT
----------------------------------------------------------------------------
0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02
----------------------------------------------------------------------------

continued: 
============================================================================
 GGA     GGC     GGG     GGT     GTA     GTC     GTG     GTT     TAC     TAT
----------------------------------------------------------------------------
0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02
----------------------------------------------------------------------------

continued: 
============================================================================
 TCA     TCC     TCG     TCT     TGC     TGG     TGT     TTA     TTC     TTG
----------------------------------------------------------------------------
0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02    0.02
----------------------------------------------------------------------------

continued: 
====
 TTT
----
0.02
----

### Specifying a parameter as constant

This means the parameter will not be modified during likelihood maximisation. We show this here by making the ``omega`` parameter constant at the value 1 -- essentially the condition of selective neutrality.

In [11]:
from cogent3 import load_tree
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
sm = get_model("CNFGTR")
lf = sm.make_likelihood_function(tree, digits=2)
lf.set_param_rule('omega', is_constant=True)

### Providing a starting value for a parameter

This can be useful to improve performance, the closer you are to the maximum likelihood estimator the quicker optimisation will be.

In [12]:
from cogent3 import load_tree
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
sm = get_model("CNFGTR")
lf = sm.make_likelihood_function(tree, digits=2)
lf.set_param_rule('omega', init=0.1)

### Setting parameter bounds for optimisation

This can be useful for stopping optimisers from getting stuck in a bad part of parameter space. The following is for ``omega`` in a codon model. I'm also providing an initial guess for the parameter (``init=0.1``) as well as a lower bound. An initial guess that is close to the maximum likelihood estimate will speed up optimisation.

In [13]:
from cogent3 import load_tree
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
sm = get_model("CNFGTR")
lf = sm.make_likelihood_function(tree, digits=2)
lf.set_param_rule('omega', init=0.1, lower=1e-9, upper=20.0)

### Setting an upper bound for branch length

If the branch length estimates seem too large, setting just an upper bound can be sensible. This will apply to all edges on the tree.

In [14]:
from cogent3 import load_tree
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
sm = get_model("F81")
lf = sm.make_likelihood_function(tree)
lf.set_param_rule('length', upper=1.0)

### Specifying rate heterogeneity functions

We extend the simple gamma distributed rate heterogeneity case for nucleotides from above to construction of the actual likelihood function. We do this for 4 bins and constraint the bin probabilities to be equal.

In [15]:
from cogent3 import load_tree
from cogent3.evolve.models import get_model

sm = get_model("GTR", with_rate=True, distribution='gamma')
tree = load_tree('../data/primate_brca1.tree')
lf = sm.make_likelihood_function(tree, bins=4, digits=2)
lf.set_param_rule('bprobs', is_constant=True)

### Specifying Phylo-HMMs

In [16]:
from cogent3 import load_tree
from cogent3.evolve.models import get_model

sm = get_model("GTR", with_rate=True, distribution='gamma')
tree = load_tree('../data/primate_brca1.tree')
lf = sm.make_likelihood_function(tree, bins=4, sites_independent=False,
                                digits=2)
lf.set_param_rule('bprobs', is_constant=True)

### Fitting likelihood functions - Choice of optimisers

There are 2 types of optimiser: simulated annealing, a *global* optimiser; and Powell, a *local* optimiser. The simulated annealing method is slow compared to Powell and in general Powell is an adequate choice. I setup  a simple nucleotide model to illustrate these.

In [17]:
from cogent3 import load_tree, load_aligned_seqs
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
aln = load_aligned_seqs('../data/primate_brca1.fasta')
sm = get_model("F81")
lf = sm.make_likelihood_function(tree, digits=3, space=2)
lf.set_alignment(aln)
lf.optimise(show_progress=False)

The default is to use Powell. For Powell, it's recommended to set the ``max_restarts`` argument since this provides a mechanism for Powell to attempt restarting the optimisation from a slightly different spot which can help in overcoming local maxima.

In [18]:
lf.optimise(local=True, max_restarts=5, show_progress=False)

We might want to do crude simulated annealing following by more rigorous Powell. To do this we first need to use the global optimiser, setting ``local=False`` setting a large value for ``global_tolerance``.

In [19]:
lf.optimise(local=False, global_tolerance=1.0, show_progress=False)

Followed by a standard call to `optimise()`.

In [20]:
lf.optimise(show_progress=False, max_restarts=5, tolerance=1e-8)

### How to check your optimisation was successful

There is no guarantee that an optimised function has achieved a global maximum. We can, however, be sure that a maximum was achieved by validating that the optimiser stopped because the specified tolerance condition was met, rather than exceeding the maximum number of evaluations. The latter number is set to ensure optimisation doesn't proceed endlessly. If the optimiser exited because this limit was exceeded you can be sure that the function **has not** been successfully optimised.

We can monitor this situation using the ``limit_action`` argument to ``optimise``. Providing the value ``raise`` causes an exception to be raised if this condition occurs, as shown below. Providing ``warn`` (default) instead will cause a warning message to be printed to screen but execution will continue. The value ``ignore`` hides any such message.

In [21]:
from cogent3 import load_tree, load_aligned_seqs
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
aln = load_aligned_seqs('../data/primate_brca1.fasta')
sm = get_model("F81")
lf = sm.make_likelihood_function(tree, digits=3, space=2)
lf.set_alignment(aln)
try:
    lf.optimise(show_progress=False, limit_action='raise',
             max_evaluations=10, return_calculator=True)
except ArithmeticError as err:
    print(err)

FORCED EXIT from optimiser after 10 evaluations


### Overview of the fitted likelihood function

In Jupyter, the likelihood function object presents a representation of the main object features.

In [22]:
from cogent3 import load_tree, load_aligned_seqs
from cogent3.evolve.models import get_model

sm = get_model("GTR")
tree = load_tree('../data/primate_brca1.tree')
lf = sm.make_likelihood_function(tree)
aln = load_aligned_seqs('../data/primate_brca1.fasta')
lf.set_alignment(aln)
lf.optimise(local=True, show_progress=False)
lf

Likelihood function statistics
log-likelihood = -6992.7690
number of free parameters = 16
==============================================
   A/C       A/G       A/T       C/G       C/T
----------------------------------------------
1.2316    5.2534    0.9585    2.3159    5.9700
----------------------------------------------
==============================
      edge    parent    length
------------------------------
    Galago      root    0.1731
 HowlerMon      root    0.0449
    Rhesus    edge.3    0.0216
 Orangutan    edge.2    0.0077
   Gorilla    edge.1    0.0025
     Human    edge.0    0.0061
Chimpanzee    edge.0    0.0028
    edge.0    edge.1    0.0000
    edge.1    edge.2    0.0034
    edge.2    edge.3    0.0120
    edge.3      root    0.0076
------------------------------
====================================
     A         C         G         T
------------------------------------
0.3757    0.1742    0.2095    0.2406
------------------------------------

#### Log likelihood and number of free parameters

Reusing the optimised `lf` object from above, we can get the log-likelihood and the number of free parameters.

In [23]:
lnL = lf.lnL
lnL

-6992.768994254359

In [24]:
nfp = lf.nfp
nfp

16

#### Aikake Information Criterion

Reusing the optimised `lf` object from above.

In [25]:
lf.get_aic()

14017.537988508719

We can also get the second-order AIC.

In [26]:
lf.get_aic(second_order=True)

14017.732482609541

#### Bayesian Information Criterion

Reusing the optimised `lf` object from above.

In [27]:
lf.get_bic()

14112.615784311509

#### Getting maximum likelihood estimates

Reusing the optimised `lf` object from above.

##### One at a time

We get the statistics out individually. We get the ``length`` for the Human edge and the exchangeability parameter ``A/G``.

In [28]:
a_g = lf.get_param_value('A/G')
a_g

5.253427222418745

In [29]:
human = lf.get_param_value('length', 'Human')
human

0.006060124812429323

##### Just the motif probabilities

In [30]:
mprobs = lf.get_motif_probs()
mprobs

0.241,0.174,0.376,0.209


##### As tables

In [31]:
tables = lf.get_statistics(with_motif_probs=True, with_titles=True)
tables[0]  # just displaying the first

1.2316,5.2534,0.9585,2.3159,5.9700


### Testing Hypotheses - Using Likelihood Ratio Tests

We test the molecular clock hypothesis for human and chimpanzee lineages. The null has these two branches constrained to be equal.

In [32]:
from cogent3 import load_tree, load_aligned_seqs
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
aln = load_aligned_seqs('../data/primate_brca1.fasta')
sm = get_model("F81")
lf = sm.make_likelihood_function(tree, digits=3, space=2)
lf.set_alignment(aln)
lf.set_param_rule('length', tip_names=['Human', 'Chimpanzee'],
        outgroup_name='Galago', clade=True, is_independent=False)
lf.set_name('Null Hypothesis')
lf.optimise(local=True, show_progress=False)
null_lnL = lf.lnL
null_nfp = lf.nfp
lf

Null Hypothesis
log-likelihood = -7177.4403
number of free parameters = 10
==========================
      edge  parent  length
--------------------------
    Galago    root   0.167
 HowlerMon    root   0.044
    Rhesus  edge.3   0.021
 Orangutan  edge.2   0.008
   Gorilla  edge.1   0.002
     Human  edge.0   0.004
Chimpanzee  edge.0   0.004
    edge.0  edge.1   0.000
    edge.1  edge.2   0.003
    edge.2  edge.3   0.012
    edge.3    root   0.009
--------------------------
==========================
    A      C      G      T
--------------------------
0.376  0.174  0.209  0.241
--------------------------

The alternate allows the human and chimpanzee branches to differ by just setting all lengths to be independent.

In [33]:
lf.set_param_rule('length', is_independent=True)
lf.set_name('Alt Hypothesis')
lf.optimise(local=True, show_progress=False)
alt_lnL = lf.lnL
alt_nfp = lf.nfp
lf

Alt Hypothesis
log-likelihood = -7175.7756
number of free parameters = 11
==========================
      edge  parent  length
--------------------------
    Galago    root   0.167
 HowlerMon    root   0.044
    Rhesus  edge.3   0.021
 Orangutan  edge.2   0.008
   Gorilla  edge.1   0.002
     Human  edge.0   0.006
Chimpanzee  edge.0   0.003
    edge.0  edge.1   0.000
    edge.1  edge.2   0.003
    edge.2  edge.3   0.012
    edge.3    root   0.009
--------------------------
==========================
    A      C      G      T
--------------------------
0.376  0.174  0.209  0.241
--------------------------

We import the function for computing the probability of a chi-square test statistic, compute the likelihood ratio test statistic, degrees of freedom and the corresponding probability.

In [34]:
from cogent3.maths.stats import chisqprob

LR = 2 * (alt_lnL - null_lnL) # the likelihood ratio statistic
df = (alt_nfp - null_nfp) # the test degrees of freedom
p = chisqprob(LR, df)
print(f'LR={LR:.4f} ; df={df}; p={df:.4f}')

LR=3.3294 ; df=1; p=1.0000


#### Testing Hypotheses - By parametric bootstrapping

In [35]:
from cogent3 import load_tree, load_aligned_seqs
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
aln = load_aligned_seqs('../data/primate_brca1.fasta')

sm = get_model("F81")
lf = sm.make_likelihood_function(tree, digits=3, space=2)
lf.set_alignment(aln)
lf.set_param_rule('length', tip_names=['Human', 'Chimpanzee'],
        outgroup_name='Galago', clade=True, is_independent=False)
lf.set_name('Null Hypothesis')
lf.optimise(local=True, show_progress=False)
sim_aln = lf.simulate_alignment()
sim_aln[:60]

,0
Rhesus,AACATGGTAAAGGTAAACGTGGGAATTTTGCGAGGAAAGAAAGGAAAACGATGTATAACT
Chimpanzee,.................T....A...............................T.....
Galago,.T..A..C.........TAAT...........T.......G......G......T..T..
Gorilla,.................T....A...............................T.....
HowlerMon,.................T....................T...A....G......T.....
Human,.................T....A...............................T.....
Orangutan,.................T....A...............................T.....


### Determining confidence intervals on MLEs

The profile method is used to calculate a confidence interval for a named parameter. We show it here for a global substitution model exchangeability parameter (*kappa*, the ratio of transition to transversion rates) and for an edge specific parameter (just the human branch length).

In [36]:
from cogent3 import load_tree, load_aligned_seqs
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
aln = load_aligned_seqs('../data/primate_brca1.fasta')
sm = get_model("HKY85")
lf = sm.make_likelihood_function(tree)
lf.set_alignment(aln)
lf.optimise(local=True, show_progress=False)
kappa_lo, kappa_mle, kappa_hi = lf.get_param_interval('kappa')
print(f"lo={kappa_lo:.2f} ; mle={kappa_mle:.2f} ; hi={kappa_hi:.2f}")
human_lo, human_mle, human_hi = lf.get_param_interval('length', 'Human')
print(f"lo={human_lo:.2f} ; mle={human_mle:.2f} ; hi={human_hi:.2f}")

lo=3.78 ; mle=4.44 ; hi=5.22
lo=0.00 ; mle=0.01 ; hi=0.01


### Saving results

The best approach is to use the json string from the ``to_json()`` method. The saved data can be later reloaded using ``cogent3.util.deserialise.deserialise_object()``. The ``json`` data contains the alignment, tree topology, substitution model, parameter values, etc..

To illustrate this, I create a very simple likelihood function. The ``json`` variable below is just a string that can be saved to disk.

In [37]:
from cogent3 import load_tree, load_aligned_seqs
from cogent3.evolve.models import get_model

aln = make_aligned_seqs(data=dict(a="ACGG", b="ATAG", c="ATGG"))
tree = make_tree(tip_names=aln.names)
sm = get_model("F81")
lf = sm.make_likelihood_function(tree)
lf.set_alignment(aln)
json = lf.to_json()
json[:60]  # just truncating the displayed string

'{"model": {"alphabet": {"motifset": ["T", "C", "A", "G"], "g'

We deserialise the object from the string.

In [38]:
from cogent3.util.deserialise import deserialise_object

newlf = deserialise_object(json)
newlf

unnamed
log-likelihood = -14.2727
number of free parameters = 3
========================
edge    parent    length
------------------------
   a      root    1.0000
   b      root    1.0000
   c      root    1.0000
------------------------
====================================
     A         C         G         T
------------------------------------
0.3333    0.0833    0.4167    0.1667
------------------------------------

### Reconstructing ancestral sequences

We first fit a likelihood function.

In [39]:
from cogent3 import load_tree, load_aligned_seqs
from cogent3.evolve.models import get_model

tree = load_tree('../data/primate_brca1.tree')
aln = load_aligned_seqs('../data/primate_brca1.fasta')
sm = get_model("F81")
lf = sm.make_likelihood_function(tree, digits=3, space=2)
lf.set_alignment(aln)
lf.optimise(show_progress=False)

We then get the most likely ancestral sequences.

In [40]:
ancestors = lf.likely_ancestral_seqs()
ancestors[:60]

,0
root,TGTGGCACAAATACTCATGCCAGCTCATTACAGCATGAGAACAGTTTGTTACTCACTAAA
edge.0,...............................................A............
edge.1,...............................................A............
edge.2,...............................................A............
edge.3,............................................................


Or we can get the posterior probabilities (returned as a ``DictArray``) of sequence states at each node.

In [41]:
ancestral_probs = lf.reconstruct_ancestral_seqs()
ancestral_probs["root"][:5]

0,0.182,0.000,0.000,0.000
1,0.000,0.000,0.000,0.156
2,0.182,0.000,0.000,0.000
3,0.000,0.000,0.000,0.156
4,0.000,0.000,0.000,0.156


### Tips for improved performance -- sequentially build the fitting

There's nothing that improves performance quite like being close to the maximum likelihood values. So using the ``set_param_rule`` method to provide good starting values can be very useful. As this can be difficult to do one easy way is to build simpler models that are nested within the one you're interested in. Fitting those models and then relaxing constraints until you’re at the parameterisation of interest can markedly improve optimisation speed.